In [1]:
from dotenv import load_dotenv

import os

load_dotenv()

True

## Construção de Tools 

A construção de Tools com o objeto ``Structuredtool`` é importante devido a sua capacidade de ser compatível com as cadeias do ``Langchain``. Isso promove uma série de benefícios, como suporte para ``paralelização``, ``reutilização``, ``observabilidade``, ``monitoramento`` e outros.

Para a construção das ferramentas, podemos utilizar o código abaixo, o exemplo que vamos utilizar será reutilizado para construir uma ferramenta de agente que irá pesquisar tendências via ``Google Trends`` e retornará uma lista de conteúdos que estão em alta.

In [2]:
# Importar a biblioteca necessária
from pytrends.request import TrendReq
from langchain_core.tools import StructuredTool

# Inicializar um objeto TrendReq
pytrends = TrendReq(hl='pt-BR', tz=360)

def trends_per_country(country: str, head: int) -> str:
    """trends per country"""
    country = country.lower()
    daily_trending_searches = pytrends.trending_searches(pn=country)
    word_list = daily_trending_searches.head(head).values.T[0]
    return "\n".join(word_list)


async def atrends_per_country(country: str, head: int) -> str:
    """trends per country"""
    country = country.lower()
    daily_trending_searches = pytrends.trending_searches(pn=country)
    word_list = daily_trending_searches.head(head).values.T[0]
    return "\n".join(word_list)

def create_trends_tool():
    return StructuredTool.from_function(func=trends_per_country, coroutine=atrends_per_country)

trends = create_trends_tool()

print(trends.invoke({"country": "brazil", "head": 10}))
print("--------------------------------")
print(
    await trends.ainvoke({"country": "united_states", "head": 10})
)  # Uses use provided amultiply without additional overhead


IBFC Correios
André Feldman
UFC
Al Nassr
Eintracht x Borussia
Roma x Genoa
Benfica x Famalicão
VALE3
Univ de Chile x River Plate
Vasco Copinha
--------------------------------
UFC
Kristi Noem
Equal Rights Amendment
Dodgers
Mac Miller
Back in Action
Joan Plowright
Opera
Edge
Unrivaled basketball


## Construção de Tools com o Serper

Aqui vamos construir outra ferramenta que irá pesquisar as tendências geradas pelo pytrends da etapa anterior. Abaixo você vai encontrar uma função que recebe uma lista de queries e retorna uma lista de notícias que foram retornadas pelo ``GoogleSerperAPIWrapper`` (Pesquise sobre o Serper para entender melhor).

Por fim, vamos acompanhar o código que será utilizado para construir a ferramenta.

In [3]:
from typing import List
import os

from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_community.document_loaders import WebBaseLoader
from tqdm import tqdm

def get_serper_results(queries: List[str],
                       k: int=5,
                       type_content: str='news',
                       hl: str='pt',
                       gl: str='br') -> List[dict]:
    
    """
    Get the multiple search results from Google Serper API
    
    Args:
    - queries: List of queries to be searched
    - k: Number of results to be returned
    - type_content: Type of content to be searched
    - hl: Language to be searched
    - gl: Country to be searched
    
    Returns:
    - List of results from the search
    
    """

    search = GoogleSerperAPIWrapper(gl=gl,
                                    hl=hl,
                                    k=k, 
                                    type=type_content) # type: Literal['news', 'search', 'places', 'images'] = 'search'
    
    results = [search.results(query) for query in queries]
    return results

def get_serper_with_scrapping(queries: List[str],
                              k: int=5,
                              type_content: str='news',
                              hl: str='pt',
                              gl: str='br') -> List[dict]:
    
    """
    Get the multiple search results from Google Serper API and scrap the content
    
    Args:
    - queries: List of queries to be searched
    - k: Number of results to be returned
    - type_content: Type of content to be searched
    - hl: Language to be searched
    - gl: Country to be searched
    
    Returns:
    - List of results from the search
    
    """
    requests_kwargs = {
    'timeout': 5  # Define o timeout para 5 segundos
    }
    
    results = get_serper_results(queries, 
                                 k, 
                                 type_content, 
                                 hl, 
                                 gl)
    
    dict_results_news = {}
    for r in tqdm(results):
        q = r['searchParameters']['q']
        news = r['news']
        
        for i in range(len(news)):
            n = news[i]
            link = n['link']
            loader = WebBaseLoader(web_paths=[link],
                                    requests_kwargs=requests_kwargs) # Adiciona timeout
            docs = loader.load()
            news[i]['content'] = "\n".join([x.page_content for x in docs])
        
        dict_results_news[q] = news
        
    return dict_results_news

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
def to_list(text):
    return text.split("\n")

trends_text = trends.invoke({"country": "brazil", 
                             "head": 3})
trends_list = to_list(trends_text)
trends_list

['IBFC Correios', 'André Feldman', 'UFC']

In [6]:
contents_news = get_serper_with_scrapping(trends_list, k=5)

100%|██████████| 3/3 [00:15<00:00,  5.31s/it]


In [7]:
contents_news

{'IBFC Correios': [{'title': '"Sem previsão", diz banca sobre resultado do concurso dos Correios',
   'link': 'https://www.metropoles.com/brasil/sem-previsao-diz-banca-sobre-resultado-do-concurso-dos-correios',
   'snippet': 'Concurso dos Correios foi realizado em dezembro do último ano e oferece 3,5 mil vagas de nível médio e superior.',
   'date': '3 dias atrás',
   'source': 'Metrópoles',
   'imageUrl': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQxtuFpEc5BgT-ULyTHMf5c22b_V3moofeWF2qsS8aBR-plGAj3qWzAYbk&usqp=CAI&s',
   'position': 1,
   'content': '“Sem previsão”, diz banca sobre resultado do concurso dos Correios | MetrópolesNotificaçõesMetropólesmetropoles.comAssine nossas newslettersBuscametropoles.comÚltimas notíciasBrasilDistrito FederalSão PauloMundoSaúdeVida & EstiloGrande AngularBlog do NoblatIgor GadelhaMario SabinoPaulo CappelliTácio LorranClaudia MeirelesFábia OliveiraViolência contra a mulherEntretenimentoCelebridadesGastronomiaRádio MetrópolesEsportesConcurso

In [8]:
from IPython.display import Markdown
import smtplib # Para enviar email
from email.message import EmailMessage #pra deixar no formato que envia email

from pydantic import BaseModel, Field
from typing import List

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

model_name_openai = "gpt-4o-2024-08-06" # Garante 100% do output em JSON

llm_openai = ChatOpenAI(
    model=model_name_openai, # 100% json output
    temperature=0,
)

system_prompt = """
Você é um assistente de IA muito prestativo que vai auxiliar um jornalista a classificar notícias que falam especificamente de futebol. Você precisa classificar a notícia como YES ou NO, dependendo se o texto fala de futebol ou não.
"""

prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt), 
            ("human", "query do usuário: \n\n {query}")
        ]
)

class GetSchema(BaseModel):
    """Schema de futebol"""
    
    resultado: str = Field(description="YES caso o texto fale de FUTEBOL e NO caso contrário", examples=['YES', 
                                                                                                    'NO'])

news = contents_news
noticias_futebol = {}

llm_openai_with_tools_extraction = llm_openai.bind_tools([GetSchema]) #, strict=True)
chain_openai_structured_extraction = prompt | llm_openai_with_tools_extraction
for assunto, noticias in news.items():
    for materia in noticias:
        content = (materia['content'])
        response = chain_openai_structured_extraction.invoke({"query": content})
        result = response.tool_calls[0]['args']['resultado']
        if result == "YES":
            noticias_futebol[assunto] = noticias

print(noticias_futebol)

{}


## Conclusão:

Em primeiro lugar, é possível observar a utilidade de construir ferramentas (Tools). Elas são capazes de facilitar a reutilização de código, a paralelização de tarefas, a observabilidade e o monitoramento de tarefas. Por fim, vamos utilizar essas ferramentas para construir um agente que irá pesquisar tendências via ``Google Trends`` e retornar uma lista de conteúdos que estão em alta, a partir de pesquisas na web.